In [1]:
import numpy as np
import pandas as pd

import time
import os
import platform

import bs4
import requests

import urllib
from urllib.request import urlopen

from selenium import webdriver as wd

from IPython.display import clear_output

In [2]:
# 요청 함수
def getSource(site) :
    
    import bs4
    import requests
    
    # 헤더 정보
    header_info = {
        'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWeb Kit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.146 Safari/537.36'
    }
    
    # 요청한다.
    response = requests.get(site, headers=header_info)
    
    # bs4 객체 생성
    soup = bs4.BeautifulSoup(response.text, 'lxml')
    
    return soup

In [3]:
# 한 페이지에 있는 다음 뉴스 링크 수집 함수
def getNewsLink(soup):

    link_list = []

    # li 태그 가져오기
    a1 = soup.select('#clusterResultUL > li')
    # print(len(a1))
    
    for a2 in a1:
        
        # div 태그 가져오기
        a3 = a2.select_one('div.wrap_cont > div > div > a')
        # print(a3)
        
        # 기사링크 
        data1 = a3.attrs['href']
        # print(data1)
        
        # 기사제목
        data2 = a3.text.strip()
        # print(data2)
    
        # span 태그 가져오기
        a4 = a2.select_one('div.wrap_cont > div > span.f_nb.date')
        a5 = a4.text.strip().split('|')
        
        # 날짜
        data3 = a5[0]
        
        # 언론사
        data4 = a5[1]
        
        # print(data1, data2, data3, data4)
        
        # 기사 링크 리스트에 저장
        link_list.append(data1)
    
    # 데이터프레임 생성
    df1 = pd.DataFrame(link_list)
    display(df1)
    
#     FILENAME = 'chosun_link.csv'
#     FILENAME = 'joongang_link.csv'
#     FILENAME = 'donga_link.csv'
#     FILENAME = 'jtbc_link.csv'
    FILENAME = 'khan_link.csv'
#    FILENAME = 'hani_link.csv'
    if os.path.exists(FILENAME) == False:
        # 파일이 없을 경우
        df1.to_csv(FILENAME, encoding='utf-8-sig', index=False)
    else:
        # mode='a' : 기존 것 뒤에다 붙여줌
        df1.to_csv(FILENAME, encoding='utf-8-sig', index=False, header=False, mode='a')
    
        
    return link_list

In [4]:
# 다음 페이지 존재 여부 확인하는 함수
def getNextPage(site) :
    
    # url에서 p= 값 들고오기
    p = site.split('&')[-1].split('=')[-1]
    
    # p값에 1 더해서 다음 페이지 url 만들기
    nextPage = site[:-len(p)] + str(int(p)+1)
    
#     print(len(p))
#     print(site)
#     print(next_page)

    # 현재 페이지와 다음 페이지 soup 가져오기
    soup1 = getSource(site)
    soup2 = getSource(nextPage)
    # print(soup1)
    # print(soup2)

    # 현재 페이지와 다음 페이지 첫번째 a 태그에서 링크 가져오기
    a1 = soup1.select('#clusterResultUL > li > div.wrap_cont > div > div > a')[0].attrs['href']
    a2 = soup2.select('#clusterResultUL > li > div.wrap_cont > div > div > a')[0].attrs['href']
    print(a1)
    print(a2)
   
 
    # 두 링크가 같지 않으면 다음 페이지가 있다고 간주, 다음 페이지 return 
    if a1 != a2 :
        return True
    # 같으면 다음 페이지 없다고 간주, False return 
    else :
        return False


In [5]:
KEYWORD = urllib.parse.quote('보궐선거')
# COLOPHON = '조선일보'
# COLOPHON = '중앙일보'
# COLOPHON = '동아일보'
# COLOPHON = 'JTBC'
COLOPHON = '경향신문'
# COLOPHON = '한겨레'

# dayStart = '20200910000000'
# dayEnd = '20210310000000'
# dayStart = '20210310000001'
# dayEnd = '20210326000000'
# 경향신문

dayStart = '20210406215601'
dayEnd   = '20210407200000'

page = 1
URL = 'https://search.daum.net/search?w=news&enc=utf8&cluster=y&cluster_page=1&'
cp_dict = {'조선일보' : '16d4PV266g2j-N3GYq',
           '중앙일보' : '16Elf9uX5H6T5xXvQV',
           '동아일보' : '16bOiOx4gG2S18EPLj',
           'JTBC'     : '16yZfDfR_rGcw5F-P0',
           '경향신문' : '16akMkKFDu6n8GTzZr',
           '한겨레' : '16nzyJHdH5ORpabfqG'}
cpName = urllib.parse.quote(COLOPHON)
cp = cp_dict[COLOPHON]

while True :
    time.sleep(1)
            
    clear_output(wait=True)
        
    site = f'{URL}q={KEYWORD}&cpname={cpName}&cp={cp}&period=6m&sd={dayStart}&ed={dayEnd}&DA=PGD&p={page}'

    print(f' 다음 뉴스 - {COLOPHON} : {page} 페이지 수집 중' )
    
    soup = getSource(site)
    getNewsLink(soup) 
    chk = getNextPage(site)

    if chk != False:
        page = page + 1
    else: 
        print('수집완료')
        break
        

 다음 뉴스 - 경향신문 : 2 페이지 수집 중


,0
0,http://v.media.daum.net/v/20210407120423460?f=o
1,http://v.media.daum.net/v/20210407110533963?f=o
2,http://v.media.daum.net/v/20210407090237847?f=o
3,http://v.media.daum.net/v/20210407112819032?f=o
4,http://v.media.daum.net/v/20210407082127524?f=o
5,http://v.media.daum.net/v/20210407030203266?f=o
6,http://v.media.daum.net/v/20210407030022167?f=o
7,http://v.media.daum.net/v/20210407000204241?f=o
8,http://v.media.daum.net/v/20210407030104197?f=o
9,http://v.media.daum.net/v/20210406220148873?f=o


http://v.media.daum.net/v/20210407120423460?f=o
http://v.media.daum.net/v/20210407120423460?f=o
수집완료
